In [1]:
import pandas as pd

In [2]:
propiedades = pd.read_csv('propiedades_completo.csv', low_memory=False)

In [3]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338158 entries, 0 to 338157
Data columns (total 30 columns):
Unnamed: 0                    338158 non-null int64
country_name                  338157 non-null object
created_on                    338158 non-null object
currency                      289613 non-null object
description                   338144 non-null object
expenses                      36702 non-null float64
extra                         1 non-null object
floor                         31796 non-null float64
geonames_id                   287419 non-null float64
id                            338157 non-null object
image_thumbnail               332554 non-null object
lat                           245153 non-null float64
lat-lon                       245153 non-null object
lon                           245153 non-null float64
operation                     338158 non-null object
place_name                    338096 non-null object
place_with_parent_names       338158 non-nul

In [4]:
propiedades = propiedades.loc[propiedades.state_name.str.contains('G.B.A') | propiedades.state_name.str.contains('Capital Federal')]
propiedades['created_on'] = pd.to_datetime(propiedades['created_on'])
propiedades.insert(3, 'year', propiedades['created_on'].dt.year)
propiedades['year'] = propiedades['year'].fillna(0).astype(int)
propiedades = propiedades.loc[propiedades.year > 2012]
propiedades = propiedades.drop(propiedades[['Unnamed: 0','country_name','lat-lon','properati_url','image_thumbnail','surface_in_m2','title','extra']], axis=1)



In [5]:
def priceFromM2(row):
    if(np.isnan(row['price_aprox_usd']) and not np.isnan(row['price_usd_per_m2']) and not np.isnan(row['surface_total_in_m2'])):
        return (row['price_usd_per_m2'] * row['surface_total_in_m2'])
    return row['price_aprox_usd']

def priceToUsd(row):
    if(np.isnan(row['price_aprox_usd']) and (row['price'] != row['price_aprox_usd'])):
        return row['price_aprox_usd']
    return row['price']

def totalM2toAparment(row):
    if(np.isnan(row['surface_total_in_m2']) and not np.isnan(row['surface_covered_in_m2']) and (str(row['property_type']) != 'house')):
        return row['surface_covered_in_m2']
    return row['surface_total_in_m2']
    
def getPM2FromPriceAndM2(row):
    if(np.isnan(row['price_usd_per_m2']) and (not np.isnan(row['price_aprox_usd']) and not np.isnan(row['surface_total_in_m2']) and row['surface_total_in_m2'] != 0)):
        return (row['price_aprox_usd'] / row['surface_total_in_m2'])
    return row['price_usd_per_m2']

In [6]:
import numpy as np

propiedades['price'] = propiedades.apply(lambda row: priceToUsd(row), axis=1)
propiedades['surface_total_in_m2'] = propiedades.apply(lambda row: totalM2toAparment(row), axis=1)
propiedades['price_usd_per_m2'] = propiedades.apply(lambda row: getPM2FromPriceAndM2(row), axis=1)
propiedades['price_aprox_usd'] = propiedades.apply(lambda row: priceFromM2(row), axis=1)

In [7]:
propiedades.to_csv('propiedades-clean.csv')

In [8]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221341 entries, 1 to 338156
Data columns (total 23 columns):
created_on                    221341 non-null datetime64[ns]
year                          221341 non-null int64
currency                      200125 non-null object
description                   221331 non-null object
expenses                      30133 non-null float64
floor                         24326 non-null float64
geonames_id                   176375 non-null float64
id                            221341 non-null object
lat                           155883 non-null float64
lon                           155883 non-null float64
operation                     221341 non-null object
place_name                    221279 non-null object
place_with_parent_names       221341 non-null object
price                         204141 non-null float64
price_aprox_local_currency    204141 non-null float64
price_aprox_usd               204141 non-null float64
price_per_m2                